In [15]:
import pandas as pd
import numpy as np
import sys
sys.path.append('c:\\Users\\RémiJAYLET\\OneDrive - eleven\\Documents\\GitHub\\CityAndYou-PublicDataUseCases\\utils')

In [16]:
pd.set_option('display.max_columns', None)

In [17]:
path = r"C:\Users\RémiJAYLET\Documents\City&You\0_data"

In [7]:
locaux_df = pd.read_csv(path+r"\MAJIC\locaux_selected.csv", sep=',', encoding='utf-8-sig', low_memory=False, dtype={'Département (Champ géographique)' : str,'Code Direction (Champ géographique)' : str,'Code Commune (Champ géographique)' : str,'Préfixe (Références cadastrales)' : str,'Section (Références cadastrales)' : str,'N° plan (Références cadastrales)' : str,'N° voirie (Adresse du local)' : 'Int64','Code voie MAJIC (Adresse du local)' : 'Int64'})
parcelles_df = pd.read_csv(path+r"\MAJIC\parcelles_selected.csv", sep=',', encoding='utf-8-sig', low_memory=False, dtype={'Département (Champ géographique)' : str,'Code Direction (Champ géographique)' : str,'Code Commune (Champ géographique)' : str,'Préfixe (Références cadastrales)' : str,'Section (Références cadastrales)' : str,'N° plan (Références cadastrales)' : str,'N° de voirie (Adresse parcelle)' : 'Int64','Code voie MAJIC (Adresse parcelle)' : 'Int64'})

locaux_df = locaux_df[['idParcelle', 'batimentLocal', 'entreeLocal', 'niveauLocal', 'porteLocal', 'numeroVoirieLocal', 'sirenProprietaireLocal', 'denominationProprietaireLocal']]
parcelles_df = parcelles_df[['idParcelle', 'suf', 'natureCultureSuf', 'sirenProprietaireSuf', 'denominationProprietaireSuf']]
copro_df = pd.read_csv(path+r"\rnc-data-gouv-with-qpv.csv", sep=',', dtype={'Siret représentant légal (si existe)' : str})

C:\Users\RémiJAYLET\AppData\Local\Temp\ipykernel_15308\1018721347.py:6: DtypeWarning: Columns (43,48,50) have mixed types. Specify dtype option on import or set low_memory=False.
  copro_df = pd.read_csv(path+r"\rnc-data-gouv-with-qpv.csv", sep=',', dtype={'Siret représentant légal (si existe)' : str})


In [18]:
copro_df

,EPCI,Commune,Numéro d'immatriculation,Type de syndic : bénévole / professionnel / non connu,Identification du représentant légal (raison sociale et le numéro SIRET du syndic professionnel ou Civilité/prénom/ nom du syndic bénévole ou coopératif),Siret représentant légal (si existe),Code APE,Commune du représentant légal,Mandat en cours dans la copropriété,Nom d’usage de la copropriété,Adresse de référence,Numéro et Voie (adresse de référence),Code postal (adresse de référence),Commune (adresse de référence),Adresse complémentaire 1,Adresse complémentaire 2,Adresse complémentaire 3,Nombre d'adresses complémentaires,long,lat,Date du règlement de copropriété,Résidence service,Syndicat coopératif,Syndicat principal ou syndicat secondaire,"Si secondaire, n° d’immatriculation du principal",Nombre d’ASL auxquelles est rattaché le syndicat de copropriétaires,Nombre d’AFUL auxquelles est rattaché le syndicat de copropriétaires,Nombre d’Unions de syndicats auxquelles est rattaché le syndicat de copropriétaires,Nombre total de lots,"Nombre total de lots à usage d’habitation, de bureaux ou de commerces",Nombre de lots à usage d’habitation,Nombre de lots de stationnement,Nombre d'arrêtés relevant du code de la santé publique en cours,Nombre d'arrêtés de péril sur les parties communes en cours,Nombre d'arrêtés sur les équipements communs en cours,Période de construction,Référence Cadastrale 1,Code INSEE commune 1,Préfixe 1,Section 1,Numéro parcelle 1,Référence Cadastrale 2,Code INSEE commune 2,Préfixe 2,Section 2,Numéro parcelle 2,Référence Cadastrale 3,Code INSEE commune 3,Préfixe 3,Section 3,Numéro parcelle 3,Nombre de parcelles cadastrales,nom_qp,code_qp,Copro dans ACV,Copro dans PVD
0,246900724.0,69094,AA0002402,non connu,non connu,NaN,non connu,non connu,Pas de mandat en cours,LES JARDINS DE JUSTINE,1 r des forges 69290 Grézieu-la-Varenne,1 r des forges,69290.0,Grézieu-la-Varenne,NaN,NaN,NaN,0.0,4.688192,45.746802,2005-03-08,non,non,oui,NaN,0.0,0.0,0.0,202.0,67.0,67.0,72.0,0.0,0.0,0.0,DE_2001_A_2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Non,Non
1,200046977.0,69385,AA0002576,professionnel,JOSEPH BAUR IMMOBILIER 34473827300012,34473827300012,6832A,LYON 5EME,Mandat en cours,15 RUE DE TRION,15 r de trion 69005 Lyon,15 r de trion,69005.0,Lyon,NaN,NaN,NaN,0.0,4.817259,45.756287,1988-12-08,non,non,oui,NaN,0.0,0.0,0.0,16.0,9.0,7.0,0.0,0.0,0.0,0.0,AVANT_1949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Non,Non
2,200046977.0,69381,AA0002659,professionnel,JOSEPH BAUR IMMOBILIER 34473827300012,34473827300012,6832A,LYON 5EME,Mandat en cours,20 RUE BODIN,20 r bodin 69001 Lyon,20 r bodin,69001.0,Lyon,NaN,NaN,NaN,0.0,4.834939,45.772860,1981-09-30,non,non,oui,NaN,0.0,0.0,0.0,23.0,19.0,17.0,0.0,0.0,0.0,0.0,AVANT_1949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Non,Non
3,200046977.0,69266,AA0002733,professionnel,JOSEPH BAUR IMMOBILIER 34473827300012,34473827300012,6832A,LYON 5EME,Mandat en cours,20 RUE MARTERET,20 r marteret 69100 Villeurbanne,20 r marteret,69100.0,Villeurbanne,NaN,NaN,NaN,0.0,4.871857,45.776644,1996-07-15,non,non,oui,NaN,0.0,0.0,0.0,9.0,6.0,5.0,1.0,0.0,0.0,0.0,DE_1975_A_1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Non,Non
4,200046977.0,69244,AA0002816,professionnel,JOSEPH BAUR IMMOBILIER 34473827300012,34473827300012,6832A,LYON 5EME,Mandat en cours,LE MOONLIGHT,11 av du nord 69160 Tassin-la-Demi-Lune,11 av du nord,69160.0,Tassin-la-Demi-Lune,NaN,NaN,NaN,0.0,4.776933,45.764967,2007-06-13,non,non,oui,NaN,0.0,0.0,0.0,80.0,38.0,37.0,42.0,0.0,0.0,0.0,DE_2001_A_2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Non,Non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550853,NaN,29155,AI0793273,non connu,non connu,NaN,non 

In [8]:
coproprietes_df = copro_df[["Numéro d'immatriculation", "Siret représentant légal (si existe)", "Nom d’usage de la copropriété", "Adresse de référence",
                            "Numéro et Voie (adresse de référence)",
                            "Référence Cadastrale 1", "Référence Cadastrale 2", "Référence Cadastrale 3", "Nombre de parcelles cadastrales"]]
def combiner_lignes(row):
    return [row['Référence Cadastrale 1'], row['Référence Cadastrale 2'], row['Référence Cadastrale 3']]
coproprietes_df['idCadastre'] = coproprietes_df.apply(combiner_lignes, axis=1)

C:\Users\RémiJAYLET\AppData\Local\Temp\ipykernel_15308\2774055568.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coproprietes_df['idCadastre'] = coproprietes_df.apply(combiner_lignes, axis=1)


In [9]:
coproprietes_df = coproprietes_df.drop(['Référence Cadastrale 1', 'Référence Cadastrale 2', 'Référence Cadastrale 3'], axis=1).explode('idCadastre')
coproprietes_df = coproprietes_df.dropna(subset='idCadastre')
coproprietes_df.head(3)

,Numéro d'immatriculation,Siret représentant légal (si existe),Nom d’usage de la copropriété,Adresse de référence,Numéro et Voie (adresse de référence),Nombre de parcelles cadastrales,idCadastre
9,AA0003491,96550338600074,CITE JARDIN MONPLAISIR,51 av des freres lumiere 69008 Lyon,51 av des freres lumiere,1.0,691233880E310P
10,AA0004069,96550338600074,CESAR FRANCK,37/39 r maryse bastie 69008 Lyon,37/39 r maryse bastie,2.0,69123388AR0094
10,AA0004069,96550338600074,CESAR FRANCK,37/39 r maryse bastie 69008 Lyon,37/39 r maryse bastie,2.0,69123388AR0198


In [19]:
coproprietes_df

,Numéro d'immatriculation,Siret représentant légal (si existe),Nom d’usage de la copropriété,Adresse de référence,Numéro et Voie (adresse de référence),Nombre de parcelles cadastrales,idCadastre
9,AA0003491,96550338600074,CITE JARDIN MONPLAISIR,51 av des freres lumiere 69008 Lyon,51 av des freres lumiere,1.0,691233880E310P
10,AA0004069,96550338600074,CESAR FRANCK,37/39 r maryse bastie 69008 Lyon,37/39 r maryse bastie,2.0,69123388AR0094
10,AA0004069,96550338600074,CESAR FRANCK,37/39 r maryse bastie 69008 Lyon,37/39 r maryse bastie,2.0,69123388AR0198
19,AA0005629,05881167000015,LES FAIENCIERS,75 all de la grande bastide cazaulx 13011 Mars...,75 all de la grande bastide cazaulx,3.0,132118660C0050
19,AA0005629,05881167000015,LES FAIENCIERS,75 all de la grande bastide cazaulx 13011 Mars...,75 all de la grande bastide cazaulx,3.0,132118660C0053
...,...,...,...,...,...,...,...
550854,AI0794677,NaN,10 route de Philippsbourg 67110 DAMBACH,10 r de philippsbourg 67110 Dambach,10 r de philippsbourg,1.0,67083000150070
550855,AI0797498,NaN,3 rue de l'est - nomexy,3 r de l est 88440 Nomexy,3 r de l est,1.0,88327000AH0068
550856,AI0802421,NaN,IMPASSE MARCELLE MAGDINIER NORD,"65 impasse marcelle magdinier, 38370 Saint-Cla...","65 impasse marcelle magdinier,",4.0,38378000AI0008
550856,AI0802421,NaN,IMPASSE MARCELLE MAGDINIER NORD,"65 impasse marcelle magdinier, 38370 Saint-Cla...","65 impasse marcelle magdinier,",4.0,38378000AI0735


In [10]:
loc_df = locaux_df.merge(coproprietes_df,
                            how='left',
                            left_on='idParcelle',
                            right_on='idCadastre')
loc_df = loc_df[~loc_df['idCadastre'].isna()]

In [11]:
print(len(loc_df.idParcelle.unique()))
loc_df.head(3)

26851


,idParcelle,batimentLocal,entreeLocal,niveauLocal,porteLocal,numeroVoirieLocal,sirenProprietaireLocal,denominationProprietaireLocal,Numéro d'immatriculation,Siret représentant légal (si existe),Nom d’usage de la copropriété,Adresse de référence,Numéro et Voie (adresse de référence),Nombre de parcelles cadastrales,idCadastre
53,01004000AH0541,A,02,0,1001,102,444608442,ENEDIS,AF4240859,75920075100130,L'ESSEN'CIEL,84 r des mouettes 1500 Ambérieu-en-Bugey,NaN,2.0,01004000AH0541
409,01004000AO0305,A,01,0,2001,2,444608442,ENEDIS,AE4567723,43422078600017,LION D'Or,3 pl du champ de mars 1500 Ambérieu-en-Bugey,NaN,1.0,01004000AO0305
410,01004000AO0305,A,01,0,2001,2,444608442,ENEDIS,AE4567723,43422078600017,LION D'Or,3 pl du champ de mars 1500 Ambérieu-en-Bugey,NaN,1.0,01004000AO0305


In [12]:
#loc_df[loc_df.idParcelle.duplicated()]
loc_df[loc_df.idParcelle=='97611000AE0139']

,idParcelle,batimentLocal,entreeLocal,niveauLocal,porteLocal,numeroVoirieLocal,sirenProprietaireLocal,denominationProprietaireLocal,Numéro d'immatriculation,Siret représentant légal (si existe),Nom d’usage de la copropriété,Adresse de référence,Numéro et Voie (adresse de référence),Nombre de parcelles cadastrales,idCadastre
5488245,97611000AE0139,A,01,0,1143,1,330176470,BANQUE FRANCAISE COMMERCIALE OCEAN INDIEN,AH3850476,53928431500030,ESPACE CANOPIA,LOT 47 LOTISSEMENT LES HAUTS VALLONS,NaN,3.0,97611000AE0139
5488246,97611000AE0139,C,01,0,1132,1,330176470,BANQUE FRANCAISE COMMERCIALE OCEAN INDIEN,AH3850476,53928431500030,ESPACE CANOPIA,LOT 47 LOTISSEMENT LES HAUTS VALLONS,NaN,3.0,97611000AE0139
5488247,97611000AE0139,C,01,0,1133,1,330176470,BANQUE FRANCAISE COMMERCIALE OCEAN INDIEN,AH3850476,53928431500030,ESPACE CANOPIA,LOT 47 LOTISSEMENT LES HAUTS VALLONS,NaN,3.0,97611000AE0139
5488248,97611000AE0139,C,01,0,1134,1,330176470,BANQUE FRANCAISE COMMERCIALE OCEAN INDIEN,AH3850476,53928431500030,ESPACE CANOPIA,LOT 47 LOTISSEMENT LES HAUTS VALLONS,NaN,3.0,97611000AE0139


L'identificatio nse fait à la maille 'parcelle' --> on peut se retrouver avec plusieurs batiments, plusieurs copro dans la même parcelle i.e. différentes local

In [13]:
par_df = parcelles_df.merge(coproprietes_df,
                            how='left',
                            left_on='idParcelle',
                            right_on='idCadastre')
par_df = par_df[~par_df['idCadastre'].isna()]

In [14]:
print(len(par_df.idParcelle.unique()))
par_df.head(3)

5554


,idParcelle,suf,natureCultureSuf,sirenProprietaireSuf,denominationProprietaireSuf,Numéro d'immatriculation,Siret représentant légal (si existe),Nom d’usage de la copropriété,Adresse de référence,Numéro et Voie (adresse de référence),Nombre de parcelles cadastrales,idCadastre
4930,01053000AD0468,NaN,Sols,552052938,LE VETEMENT MODERNE,AH5046131,85020985900022,10 AVENUE ALSACE LORRAINE,10 av alsace lorraine 1000 Bourg-en-Bresse,NaN,1.0,01053000AD0468
5042,01053000AV0038,NaN,Sols,552081317,ELECTRICITE DE FRANCE,AA9270208,NaN,Les Acacias,11 r de cuiron 1000 Bourg-en-Bresse,NaN,2.0,01053000AV0038
5043,01053000AV0038,NaN,Sols,552081317,ELECTRICITE DE FRANCE,AA9270208,NaN,Les Acacias,11 r de cuiron 1000 Bourg-en-Bresse,NaN,2.0,01053000AV0038
